In [2]:
import tensorflow as tf 
import numpy as np

# 说明

tf中的**变量**指的是：在训练过程中，不断在随机梯度下降中被改变的那些量！—— 会有专门的模块、函数来收集、处理这些变量！
- 创建张量（常量）：tf.constant()
- 创建变量（可变）：tf.Variabel()

好好区别张量的不变与变量的可变：

In [3]:
t = tf.constant( [2,3] )  # t若不是重新赋值，是不会改变的！
v = tf.Variable( [2,3] )

In [4]:
v.assign_add( [2,3] )  # 而变量v是可以改变数值的（同一个变量地址）！
v

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([4, 6])>

# 变量的一些操作

下面是一些常见的关于变量的操作：
- x.assign(y)：将变量x的值改变成y
- x.assign_add(y)：将变量x的值变为x+y
- x.read_value()：读取当前变量x的值 —— 因为它一直在变，故可以如此来查看
- 保存网络中的值，实质就是保存网络中随着训练变化的那些变量的值 —— 后面会讲如何保存

# 变量的自动微分计算

变量是如何变化的？ —— 随机梯度下降中，变量是沿着其梯度最大的方向变化的！
- tf.GradientTape()：自动记录变量的变化 —— 其标准写法就是放在with语句中，其中命令块放待求微分的函数式！
- 注意1：不管是变量还是常量，在求自动微分运算中，其“**值**”必须是**浮点型**的！
- 注意2：tf.GradientTape()里面的参数若不**设置一直存储（persistent = True）**，则求完一次微分后，tf.GradientTape()磁带会自动释放，即不能再求第二次！—— 若不设置，会报Runtime的错误。

In [9]:
# 对于“变量”的自动微分运算：下面是tf.GradientTape的标准写法 —— with语句是一个上下文管理器
w = tf.Variable(1.5)  # 数值必须是浮点型！ 

with tf.GradientTape() as t:  # tf.GradientTape()会一直跟踪记录变量t
    loss = w * w + 1 / w      # 待求微分的函数式
    
grad = t.gradient(loss, w)  # 前一个参数是待求微分的函数名，后一个参数是对谁求微分
grad

<tf.Tensor: id=50, shape=(), dtype=float32, numpy=2.5555556>

In [10]:
# 对于“常量”的自动微分运算：多一个watch！
w1 = tf.constant(3.0)  # 数值必须是浮点型！

with tf.GradientTape() as t:
    t.watch(w1)
    loss = w1 * w1 + 1 / w1
    
grad1 = t.gradient(loss, w1)
grad1

<tf.Tensor: id=72, shape=(), dtype=float32, numpy=5.888889>

In [11]:
# 想多次求微分的设置：变量、常量均适用
w2 = tf.Variable(3.2)

with tf.GradientTape( persistent = True ) as t:  # 持续存储设置打开！
    loss1 = w2 * w2 + 1 / w2
    loss2 = loss1 * loss1    # 多个函数 
    
grad1 = t.gradient( loss1, w2 )
grad2 = t.gradient( loss2, w2 )

In [12]:
grad1

<tf.Tensor: id=104, shape=(), dtype=float32, numpy=6.302344>

In [13]:
grad2

<tf.Tensor: id=124, shape=(), dtype=float32, numpy=133.01097>